In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tkinter as tk


In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) # Draw pose connections

In [5]:
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=5, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=5, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return np.concatenate([pose])

In [7]:
# Path for exported data, numpy arrays
#DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['not skip', 'not_skip', 'skip'])

# Thirty videos worth of data
no_sequences = 64

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
label_map = {label:num for num, label in enumerate(actions)}

In [9]:
label_map

{'not skip': 0, 'not_skip': 1, 'skip': 2}

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard

In [12]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [13]:
del model

NameError: name 'model' is not defined

In [14]:
model = Sequential()
model.add(LSTM(units = 100, input_shape = (30, 132)))
model.add(Dropout(0.2))
model.add(Dense(100,activation='relu'))
model.add(Dense(actions.shape[0], activation="softmax"))

In [15]:
model.load_weights('pakeiniyak.h5')

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [17]:
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv('skispset3.csv')
df

,waktu,loncat,kalori
0,6,11,1
1,7,13,1
2,9,19,1
3,12,26,2
4,13,29,2
...,...,...,...
154,583,1355,108
155,589,1366,109
156,591,1371,110
157,598,1390,111


In [19]:
reg = linear_model.LinearRegression()
reg.fit(df.drop('kalori',axis='columns').values,df.kalori)

LinearRegression()

In [20]:
def extract_decimal(number):
    decimal_part = number - int(number)
    return round(decimal_part, len(str(decimal_part).split('.')[1]))

In [21]:
def calpred_MET(counter,time):
    if(time <= 60):
        avg = counter
        min = time/60
    if(time > 60):
        min = time/60
        avg = counter / min
    if(avg <= 100):
        MET = 8.8
    if(avg > 100 and avg < 120):
        MET = 11.8
    if(avg >= 120):
        MET = 12.3
    print(avg)
    print(min)
    print(MET)
    pred = (MET * 3.5 * 70) / 200 
    pred = pred * min
    print(pred)
    return pred

In [22]:
def calPoly(counter,time):
    y_data = [11.025,13.475,14.3325,15.18,15.3125]
    x_data = [50,80,125,135,145]
    mymodel = np.poly1d(np.polyfit(x_data,y_data,3))
    myline = np.linspace(50, 145, 100)
    
    if(time < 60):
        avg = counter
        min = time/60
    if(time >= 60):
        min = time / 60
        avg = counter/min
    
    speed = mymodel(avg)*min
    return speed

In [27]:
def display(counter,time,calpred,calPoly):
    root = tk.Tk()
    root.title("Result")
    # Set the window size
    root.geometry("500x400")  # Width x Height
    root.eval('tk::PlaceWindow . center')
    # Set the background color of the window
    root.configure(bg="black")

    # Create a label widget with black text on a white background
    label = tk.Label(root, text=f"Jump Count: {counter} \nTime: {time} \nCalories(LR):{calpred} \nCalories(MET):{calPoly}", fg="white", bg="black",font=("Arial", 14, "bold"), justify="left")
    label.pack(anchor="w", padx=10, pady=10)

    root.mainloop()

# BUAT VIDEO INPUT

In [30]:
# 1. New detection variables
sequence = []
threshold = 0.75
thresholdj = 0.008
highest = 0
lowest=0
#UNTUK COUNTING
check1=False
check2=False
check3=False
lcheck= True
leg = None
counter=0
timer =0
pose = mp.solutions.pose.Pose()
cap = cv2.VideoCapture('jevontesting.mp4')

imsize = (1800,920)

calval =0
xlab=0
xlabel = []
ylabel =[]
lowestlabel =[]
xlowestlabel =[]
highestlabel =[]
xhighestlabel =[]
timecheck =0
#UNTUK TIMER
prev = time.time()
current_time =0
at= 0
######
start_timer = time.time()
countdown_duration = 5
checker = 0
timer_ended = False
# Set mediapipe model 
while cap.isOpened():

        # Read feed
    ret, frame = cap.read()
    if not ret:
        break
    
        # Make detections
    image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
    draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[-30:]
    pred = ' '  
    conf = ' '
    act = ' '
    try:
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            #print(res[np.argmax(res)])
            act = str(res[np.argmax(res)])
            if res[np.argmax(res)] > threshold: 
                pred = str(actions[np.argmax(res)])
                conf=str(res[np.argmax(res)])
                #print(pred)
            else :
                pred = ' '
        current_position = cap.get(cv2.CAP_PROP_FPS)
        
        
        timecheck=timecheck+1
        elapsed_timer = time.time() - start_timer
        remaining_timer = countdown_duration - elapsed_timer
        seconds = int(remaining_timer % 60)
        timer_text = "{:2d}".format(seconds)
        if remaining_timer < 0 and not timer_ended:
            
            print("Countdown timer has ended!")
            timer_ended = True
        print(checker)
        if pred == 'skip':
            checker = 0
        else:
            checker = 1
        if pred == 'skip' and leg is not None and results.pose_landmarks.landmark[24].visibility > 0.9 and timer_ended and checker ==0 :
            now=time.time()
            xlab = xlab + 1
            xlabel.append(xlab)
            ylabel.append(results.pose_landmarks.landmark[24].y)
            if leg > results.pose_landmarks.landmark[24].y:
                check1=True
                if lcheck == True:
                    lowest = leg
                    lcheck=False
                    lowestlabel.append(lowest)
                    xlowestlabel.append(xlab)
                    
            if check1==True and leg < results.pose_landmarks.landmark[24].y:
                
                highest =leg
                check2=True
                highestlabel.append(highest)
                xhighestlabel.append(xlab)
                if lowest-highest > thresholdj:
                    check3=True
                #print(results.pose_landmarks.landmark[24].y-highest)
                #print(counter)
            if check2==True and check3==True and results.pose_landmarks.landmark[24].y-highest > thresholdj:
                counter=counter+1
                lcheck = True
                check1=False
                check2=False
                check3=False
            if timecheck > current_position and pred == 'skip':
                current_time=current_time + 1
                timecheck=0
            
        
        calpred = reg.predict([[current_time, counter]])
        if calpred[0] < 0:
            calpred[0] = 0
        calval= "{:.2f}".format(calpred[0])
        
        
        
        #print(highest)
        leg = results.pose_landmarks.landmark[24].y
        #print(results.pose_landmarks.landmark[24].visibility)
       
        z = '%.2f'%leg

        image = cv2.resize(image,(imsize[0],imsize[1]))
        cv2.putText(image,'Action : ' + pred + ' Confidence : '+ conf, (50,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Jumpcount : ' + str(counter), (50,60),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Exercise Time : ' +str(current_time), (50,90),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Calories Burned : ' + str(calval), (50,120),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        if not timer_ended:
            cv2.putText(image, timer_text, (int(imsize[0]/2-50),int(imsize[1]/2)), cv2.FONT_HERSHEY_SIMPLEX, 3,(255,255,0),5)
        # Show to screen
        
        cv2.imshow('OpenCV Feed', image)
    except AttributeError:
        print("ggwp dah bang")
        # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
display(counter,current_time,calval,calPoly(counter,current_time))

0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1/1 [==============================] - 2s 2s/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 43ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 19ms/step
Countdown timer has ended!
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 18ms/step
1
1/1 [==============================] - 0s 19ms/step
1
1/1 [==============================] - 0s 20ms/step
1
1/1 [==============================] - 0s 19ms/step
1

1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 19ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [=======================

# BUAT CAMERA INPUT

In [ ]:
# 1. New detection variables
sequence = []
threshold = 0.9
thresholdj = 0.009
highest = 0
lowest=0
#UNTUK COUNTING
checker =0
check1=False
check2=False
check3=False
lcheck= True
leg = None
counter=0
test_timer =0
elapsed_time =0
timer =0
pose = mp.solutions.pose.Pose(min_detection_confidence = 0.8)
cap = cv2.VideoCapture(0)


imsize = (1280,720)

#UNTUK TIMER
prev = time.time()
current_time =0
at= 0
######
start_timer = time.time()
countdown_duration = 7
timer_ended = False
# Set mediapipe model 
while cap.isOpened():

        # Read feed
    ret, frame = cap.read()
    if not ret:
        continue
    
        # Make detections
    image, results = mediapipe_detection(frame, pose)
        
        # Draw landmarks
    draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
    keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
    sequence.append(keypoints)
    sequence = sequence[-30:]
    pred = ' '  
    conf = ' '
    act = ' '
    try:
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            #print(res[np.argmax(res)])
            act = str(res[np.argmax(res)])
            if res[np.argmax(res)] > threshold: 
                pred = str(actions[np.argmax(res)])
                conf=str(res[np.argmax(res)])
                #print(pred)
        
        
        
        elapsed_timer = time.time() - start_timer
        remaining_timer = countdown_duration - elapsed_timer
        seconds = int(remaining_timer % 60)
        timer_text = "{:2d}".format(seconds)
        if remaining_timer < 0 and not timer_ended:
            
            print("Countdown timer has ended!")
            timer_ended = True
        print(checker)
        if pred == 'skip':
            checker = 0
        else:
            checker = 1
        if pred == 'skip' and leg is not None and results.pose_landmarks.landmark[24].visibility > 0.9 and timer_ended and checker == 0:
            now=time.time()
            if leg > results.pose_landmarks.landmark[24].y:
                
                check1=True
                if lcheck == True:
                    lowest = results.pose_landmarks.landmark[24].y
                    lcheck=False
                    
            if check1==True and leg < results.pose_landmarks.landmark[24].y:
                
                highest =leg
                check2=True
                if lowest-highest > thresholdj:
                    check3=True
                #print(results.pose_landmarks.landmark[24].y-highest)
                #print(counter)
            if check2==True and check3==True and results.pose_landmarks.landmark[24].y-highest > thresholdj:
                counter=counter+1
                lcheck = True
                check1=False
                check2=False
                check3=False
            
            if now-prev > 1 and pred == 'skip':
                current_time=current_time + 1
                prev=now
            
        calpred = reg.predict([[current_time, counter]])
        calval= "{:.2f}".format(calpred[0])
        
        
        
        #print(highest)
        leg = results.pose_landmarks.landmark[24].y
        #print(results.pose_landmarks.landmark[24].visibility)
        #print(current_time)
        z = '%.2f'%leg
        timers = '{:.2f}'.format(elapsed_time)
        image = cv2.resize(image,(imsize[0],imsize[1]))
        cv2.putText(image,'Action : ' + pred + ' Confidence : '+ conf, (50,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Jumpcount : ' + str(counter), (50,60),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Exercise Time : ' +str(current_time), (50,90),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.putText(image,'Calories Burned : ' + str(calval), (50,120),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        if not timer_ended:
            cv2.putText(image, timer_text, (int(imsize[0]/2-50),int(imsize[1]/2)), cv2.FONT_HERSHEY_SIMPLEX, 3,(255,255,0),5)
        # Show to screen
        
        cv2.imshow('OpenCV Feed', image)
    except AttributeError:
        print("ggwp dah bang")
        # Break gracefully
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

display(counter,current_time,calval)

0
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1
ggwp dah bang
1/1 [==============================] - 0s 24ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 23ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 23ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 29ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 31ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 25ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 23ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 26ms/step
1
ggwp d

1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 25ms/step
1
1/1 [==============================] - 0s 26ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 25ms/step
1
1/1 [==============================] - 0s 25ms/step
1
1/1 [==============================] - 0s 25ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 26ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 27ms/step
1
1/1 [==============================] - 0s 27ms/step
1
1/1 [==============================] - 0s 26ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 21ms/step
1
ggwp dah bang
1/1 [==============================] - 0s 22ms/step
1
ggwp dah bang


0
1/1 [==============================] - 0s 26ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [=====================